In [1]:
import folium
import pandas as pd
import geopandas as gpd
import pickle

In [ ]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
orthophoto_dir = config['data']['orthophotos']
predictions_dir = config['data']['segmentation_output']
nutzungsdaten_dir = config['data']['nutzungsdaten']


In [ ]:
with open(config['data']['data'] + '/uhi_model/raw/input.pkl', 'rb') as file:
    gdf = pickle.load(file)

In [ ]:
gdf['geometry_4326'] = gdf.geometry.to_crs('EPSG:4326')

In [13]:
from shapely.geometry import box
bbox = box(*[11.539850,48.133244,11.582079,48.146990])
subset = gdf[gdf.geometry_4326.intersects(bbox)]

In [14]:
#gdf.geometry = gdf.geometry.to_crs('EPSG:4326')
styles = {
    'ignore':{"fillColor": "#000000", 'color':'#000000', 'weight':0, 'fillOpacity':0.5}, 
    'impervious':{"fillColor": "#ffffff", 'color':'#ffffff', 'weight':0, 'fillOpacity':0.5}, 
    'building':{"fillColor": "#ff00ff", 'color':'#ff00ff', 'weight':0, 'fillOpacity':0.5}, 
    'low vegetation':{"fillColor": "#00ff00", '#00ff00':'pink', 'weight':0, 'fillOpacity':0.5}, 
    'water':{"fillColor": "#0000ff", 'color':'#0000ff', 'weight':0, 'fillOpacity':0.5}, 
    'trees':{"fillColor": "#008200", 'color':'#008200', 'weight':0, 'fillOpacity':0.5}, 
    'road':{"fillColor": "#ff0000", 'color':'#ff0000', 'weight':0, 'fillOpacity':0.5}, 
    'train':{"fillColor": "#505050", 'color':'#505050', 'weight':0, 'fillOpacity':0.5}, 
}
m = folium.Map(location = [subset.iloc[0,:].geometry_4326.centroid.y, subset.iloc[0,:].geometry_4326.centroid.x], zoom_start = 14)
# Add ESRI Satellite Layer
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)
# Add Semantic Segmentation Layer
for label in subset.label.unique():
    folium.GeoJson(subset[subset.label == label].geometry_4326, style_function=lambda x, label=label: styles[label], name=label).add_to(m)
folium.LayerControl().add_to(m)
m.save('Predictions.html')
#m
